In [1]:
from num2fixed import dec_to_bin, dec_to_bin_array

In [84]:
def unsigned_fixed_add(x1, x2):
    """ unsigned fixed point add
    x1, x2: string of signed fixed point number,
            they must have the same length, e.g. 15.
    return:
        a tuple (carry_flag, fixed_result)
        carry_flag: 0 or 1
        fixed_result: string which has same bit as inputs
    """
    assert len(x1) == len(x2)
    carry = 0
    fixed_result = ''
    
    for i in reversed(range(len(x1))):
        if x1[i] == '0' and x2[i] == '0':
            if carry == 1:
                fixed_result += '1'
            else:
                fixed_result += '0'
            carry = 0
        elif x1[i] == '1' and x2[i] == '1':
            if carry == 1:
                fixed_result += '1'
            else:
                fixed_result += '0'
            carry = 1
        else: # one 1 and one 0
            if carry == 1:
                fixed_result += '0'
                carry = 1
            else:
                fixed_result += '1'
                carry = 0
    
    return (carry, fixed_result[::-1])
    
def fixed_add(x1, x2, integer_x1=8, integer_x2=8):
    """fixed point add 
    x1, x2: string of signed fixed point number 
    integer_xi: how many digits do we give to integer part
        e.g. sign 1 bit, integer 8 bits, decimal 7 bits
    return:
        a tuple (signed_fixed_result, integer_result)
        integer_result means how many digits does the integer part have
    """
    assert len(x1) == len(x2) and len(x1) == 16
    sign1 = x1[0]
    sign2 = x2[0]
    unsigned1 = x1[1:]
    unsigned2 = x2[1:]
    
    if integer_x1 > integer_x2:
        delta = integer_x1 - integer_x2
        # shift x2
        unsigned2 = fixed_shift_right(unsigned2, shift_size=delta)
    elif integer_x1 < integer_x2:
        delta = integer_x2 - integer_x1
        # shift x1
        unsigned1 = fixed_shift_right(unsigned1, shift_size=delta)
    else:
        pass
    
    integer_result = max(integer_x1, integer_x2)
    
    adjusted1 = sign1 + unsigned1
    adjusted2 = sign2 + unsigned2
    
#     print("adjusted:{}\t{}".format(adjusted1, adjusted2))
    # result is a fixed point number with a size of 17
    overflow, result = unsigned_fixed_add(adjusted1, adjusted2)
    if overflow:
        raise Exception("overflow detected")
    
    return result

def _helper_mul_add(adder_array):
    """ add an array of 63-bit fixed point number
    return:
        a 64-bit fixed point number
    """
    if len(adder_array) == 0:
        return '0' * 64
    elif len(adder_array) == 1:
        return '0' + adder_array[0]
    else:
        carry = 0
        last_ele = adder_array[0]
        
        for i in range(1, len(adder_array)):
            this_carry, temp_result = unsigned_fixed_add(adder_array[i], last_ele)
            carry += this_carry
            last_ele = temp_result
            
        if carry == 0:
            result = '0' + last_ele
        else:
            result = '1' + last_ele
        assert len(result) == 64
        
        return result

def _helper_filler1(x):
    """
    fill a 16 bit fixed point to 32 bit,
    0 or 1 depends on the sign bit
    x: input 16bit str
    return:
        a 32 bit str
    """
    if x[0] == '0':
        return 16 * '0' + x
    else: # '1'
        return 16 * '1' + x
    
def _helper_filler2(x, int_before=15, int_after=16):
    """
    fill 32 bit number to 61 bits, fill 0s
    x: 32bit str
    return:
        63 bit str
    """
    return int_before * '0' + x + int_after * '0'
    

def fixed_mul(x1, x2, integer_x1=8, integer_x2=8, integer_result=8):
    """
    fixed point multiplication
    input: x1, x2 -> 16 bit fixed point
    integer_x1 / x2 : where is the input decimal point
    integer_result: where is the result decimal point
    return:
        result -> 16 bit fixed point
    """
    adders = []
    
    fill_x1 = _helper_filler1(x1)
    fill_x2 = _helper_filler1(x2)
    
    # save all adders
    for i in range(32):
        if fill_x1[i] == '1':
            before = i
            after = 31 - i
            filled_num = _helper_filler2(fill_x2, before, after)
            adders.append(filled_num)
    
#     print("adders:")
#     for item in adders:
#         print(item)

    # add them up, a 30 bit number
    add_result = _helper_mul_add(adders)
    cut_result = add_result[32:]
#     print("add_result:", add_result)
    # pick digits
    int_part = integer_x1 + integer_x2
    new_int_start = int_part - integer_result + 1 # sign 1 bit
    
    result = cut_result[new_int_start: new_int_start + 16]
    
    return result
#     print("try:")
#     for i in range(15):
#         print(fixed2float('1' + ))
#     if (sign_x1 == '0' and sign_x2 == '0') or \
#         (sign_x1 == '1' and sign_x2 == '1'):
#         # positive
#         return '0' + num_result
#     else: # negative
#         return '1' + num_result
    
def fixed_shift_right(x, shift_size, keep_len=True):
    """
    x: string of unsigned fixed point number
    keep_len: whether maintain the original length
    return:
        shifted unsigned fixed point number (a string)
    """
    length = len(x)
    
    if keep_len:
        result = shift_size * '0' + x[shift_size:]
    else:
        result = x[shift_size:]
        
    return result
    

def relu():
    pass

def reduct_one(x):
    """given a fixed point number, reduct one from it
    x: input string of fixed point
    return:
        string of fixed point
    """
    result = '' 
    carry = 0
    
    # reduct 1 and if carry == 0, break
    for i in reversed(range(len(x))):
        if x[i] == '0':
            if carry == 0:
                result = '1' + result
                carry = 1
            else: # carry == 1
                result = '0' + result
                carry = 1
        elif x[i] == '1':
            if carry == 0:
                result = '0' + result
                # copy rest
                for j in reversed(range(i)):
                    result = x[j] + result
                break
            else: # carry == 1
                result = '0' + result
                # copy rest
                for j in reversed(range(i)):
                    result = x[j] + result
                break
#     print("result:\t\t{}".format(result))
    return result

def reverse(x):
    """given a fixed point number, reverse 0 and 1
    x: input string of fixed point
    return:
        string of fixed point
    """
    result = ''
    for i in range(len(x)):
        if x[i] == '0':
            result = result + '1'
        else:
            result = result + '0'
    
#     print("reversed:\t{}".format(result))
    return result
        
def fixed2float(x, integer_x=8):
    """fixed point to floating point
    x: string of signed fixed point number 
    integer_x: how many digits do we give to integer part
        e.g. sign 1 bit, integer 8 bits, decimal 7 bits
    return:
        a floating point number
    """
    if x[0] == '0':
        sign = 1
    else:
        sign = -1
    
#     print("x: {}".format(x))
    unsigned = 0
    shift_size = len(x) - 1 - integer_x # == demical digits number
    for i in range(len(x) - 1): # e.g. 15
        if x[i + 1] == '1': # ignore sign 
            new_fl = 2 ** (len(x) - 2 - i -shift_size)
            unsigned += new_fl
    
    if sign == 1:
        result = unsigned
    else:
        result = unsigned - 32768 * (2 ** (-shift_size))
    
    return result

In [89]:
x1 = -15.2
x2 = -3.43
fixed_x1 = dec_to_bin(x1, int_digit=8, decimal_digit=7)
fixed_x2 = dec_to_bin(x2, int_digit=8, decimal_digit=7)
print(fixed_x1, fixed_x2)
result = fixed_mul(fixed_x1, fixed_x2, integer_x1=8, integer_x2=8, integer_result=8)
print(result)
print(fixed2float(result), "actual result:", x1 * x2)

reverse:  1111100001100110
reverse:  1111111001001000
1111100001100111 1111111001001001
0001101000001110
52.109375 actual result: 52.136


## unsigned fixed add unit test

In [55]:
x1 = '010101010101010' # len = 15
x2 = '101010101010101' # len = 15

result = unsigned_fixed_add(x1, x2)
print(len(result), '\t', result)

2 	 (0, '111111111111111')


In [56]:
print(unsigned_fixed_add('0000000000000000', '1111111111111111')) # 0 + (-1)
print(unsigned_fixed_add('0000000000000100', '1111111111111111')) # 4 + (-1)

(0, '1111111111111111')
(1, '0000000000000011')


## fixed add unit test

In [57]:
x1 = '0010101010101010' # len = 15
x2 = '1001010101010101' # len = 15

result = fixed_add(x1, x2, 8, 8)
print(len(result), '\t', result)

fl_x1 = fixed2float(x1, integer_x=8)
fl_x2 = fixed2float(x2)
fl_result = fixed2float(result)
actual_result = fl_x1 + fl_x2
print("float1:\t{}\tfloat2:\t{}\tfixed_result:\t{}\tactual_result:\t{}\t".format(
        fl_x1, fl_x2, fl_result, actual_result))

16 	 1011111111111111
float1:	85.328125	float2:	-213.3359375	fixed_result:	-128.0078125	actual_result:	-128.0078125	


## fixed2float unit test

In [54]:
fixed = dec_to_bin(0.875, int_digit=8, decimal_digit=7)
fl = fixed2float(fixed, integer_x=8)

print("fixed point: {}\nfloating point: {}".format(fixed, fl))

fixed point: 0000000001110000
floating point: 0.875


In [ ]:
0 00000000 111 0000
1 11111111 000 1111
1 11111111 001 0000

In [12]:
dec_to_bin(-1, int_digit=8, decimal_digit=7)

'1111111110000000'

In [4]:
def add_1(complement):
    n = len(complement)
    carry = 1
    i = n - 1
    complement = list(complement)

    while i > 0 and carry:
        if complement[i] == '0' and carry == 1:
            complement[i] = '1'
            carry = 0
        if complement[i] == '1' and carry == 1:
            complement[i] = '0'
        i -= 1
        #print(carry)

    return "".join(complement)

In [15]:
add_1('1111111110001111')

'1111111110010000'

In [40]:
def dec_to_bin(x, int_digit, decimal_digit, decimal_point=False):
    """ given a float / int -> return string of fixed point
    
    :param x: original number
    :param int_digit: the number of digits of integer part.
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    decimal = decimal_dec_to_bin(x, decimal_digit)
    
    if x >= 0:
        sign = 1
    else:
        sign = -1
        x = -x
        
    # suppose x > 0
    integer = bin(int(x))[2:]
    if integer == "0":
        integer = ""
    add_zero_int = int_digit - len(integer)
    add_zero_integer_complement ="0" + "0" * add_zero_int + integer
        
    add_zero_dec = decimal_digit - len(decimal)
    add_zero_dec = decimal + "0" * add_zero_dec
    
    if sign == 1: # x > 0
        if decimal_point:
            return add_zero_integer_complement + "." + add_zero_dec
        else:
            return add_zero_integer_complement + add_zero_dec
    else: # x < 0
        if decimal_point:
            raise Exception("negative numbers doesn't support decimal point currently")
        else:
            temp_result = add_zero_integer_complement + add_zero_dec
            temp_result_reverse = "".join(str(0 if int(r) == 1 else 1) for r in temp_result) # radix-minus-one complement
            print("reverse: ", temp_result_reverse)
            result = add_1(temp_result_reverse)
            return result
    
def decimal_dec_to_bin(x, decimal_digit):
    """
    :param x: is the original number
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    x = abs(x)
    x -= int(x)
    bins = ""
    i = 0
    while x and i < decimal_digit:
        x *= 2
        bins += "1" if x >= 1. else "0"
        x -= int(x)
        i += 1
    return bins


 
def int_digit(x):
    """given an input x, detect how much integer digit it needs
    x: a arbitary number
    """
    if x >= 32767 or x <= -32768:
        raise Exception("digit out of range for fixed point 16")
    else:
        pass
    
    exp_range = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048,
             4096, 8192, 16384,32768]
    
    for i, r in enumerate(exp_range):
        if abs(int(x)) <= r - 1:
            return i
    raise Exception("digit out of range for fixed point 16")
    
    return 0


In [48]:
int_digit(-108.875)

7

In [53]:
num = -2242.22
int_num = int_digit(num)
fixed = dec_to_bin(num, int_digit=int_num, decimal_digit=15 - int_num)
# 1 11111111 001 0000
print(fixed2float(fixed,int_num))

reverse:  1011100111101110
-2242.125


In [6]:
bin(int(-0.875))

'0b0'

## fixed multiplication unit test